In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

So the code below should import a csv that has already been converted completely to numerical data. The commented out code was used to format the data properly.

In [58]:
df = pd.read_csv('numerical_df.csv')

In [72]:
# for index, row in df.iterrows():
#     if row['Opponent'] == 'Pep':
#         df.loc[index, 'Opponent'] = 0
#     elif row['Opponent'] == 'Stan':
#         df.loc[index, 'Opponent'] = 1
#     elif row['Opponent'] == 'UCLA':
#         df.loc[index, 'Opponent'] = 2
#     elif row['Opponent'] == 'OSU':
#         df.loc[index, 'Opponent'] = 3
# for index, row in df.iterrows():
#     if row['BYU_FL_Position'] == 'OH':
#         df.loc[index, 'BYU_FL_Position'] = 0
#     elif row['BYU_FL_Position'] == 'MB':
#         df.loc[index, 'BYU_FL_Position'] = 1
#     elif row['BYU_FL_Position'] == 'S':
#         df.loc[index, 'BYU_FL_Position'] = 2
#     elif row['BYU_FL_Position'] == 'L':
#         df.loc[index, 'BYU_FL_Position'] = 3
# for index, row in df.iterrows():
#     if row['BYU_BM_Position'] == 'OH':
#         df.loc[index, 'BYU_BM_Position'] = 0
#     elif row['BYU_BM_Position'] == 'MB':
#         df.loc[index, 'BYU_BM_Position'] = 1
#     elif row['BYU_BM_Position'] == 'S':
#         df.loc[index, 'BYU_BM_Position'] = 2
#     elif row['BYU_BM_Position'] == 'L':
#         df.loc[index, 'BYU_BM_Position'] = 3
# for index, row in df.iterrows():
#     if 'Pep' in row['OPP_BL_ID']:
#         df.loc[index, 'OPP_BL_ID'] = int(row['OPP_BL_ID'].replace('Pep', '1'))
#     elif 'Stan' in row['OPP_BL_ID']:
#         df.loc[index, 'OPP_BL_ID'] = int(row['OPP_BL_ID'].replace('Stan', '2'))
#     elif 'UCLA' in row['OPP_BL_ID']:
#         df.loc[index, 'OPP_BL_ID'] = int(row['OPP_BL_ID'].replace('UCLA', '3'))
#     elif 'OSU' in row['OPP_BL_ID']:
#         df.loc[index, 'OPP_BL_ID'] = int(row['OPP_BL_ID'].replace('OSU', '4'))
#     else:
#       pass
# for index, row in df.iterrows():
#     if row['BYU_BR_Position'] == 'OH':
#         df.loc[index, 'BYU_BR_Position'] = 0
#     elif row['BYU_BR_Position'] == 'MB':
#         df.loc[index, 'BYU_BR_Position'] = 1
#     elif row['BYU_BR_Position'] == 'S':
#         df.loc[index, 'BYU_BR_Position'] = 2
#     elif row['BYU_BR_Position'] == 'L':
#         df.loc[index, 'BYU_BR_Position'] = 3
#     elif row['BYU_BR_Position'] == 'OPP':
#         df.loc[index, 'BYU_BR_Position'] = 4
#     elif row['BYU_BR_Position'] == 'DS':
#         df.loc[index, 'BYU_BR_Position'] = 5
#     elif row['BYU_BR_Position'] == 'L/DS':
#         df.loc[index, 'BYU_BR_Position'] = 6

# for index, row in df.iterrows():
#     if row['Classification_target'] == 'FL':
#         df.loc[index, 'Classification_target'] = 0
#     elif row['Classification_target'] == 'Service Error':
#         df.loc[index, 'Classification_target'] = 1
#     elif row['Classification_target'] == 'FM':
#         df.loc[index, 'Classification_target'] = 2
#     elif row['Classification_target'] == 'FR':
#         df.loc[index, 'Classification_target'] = 3
#     elif row['Classification_target'] == 'BL':
#         df.loc[index, 'Classification_target'] = 4
#     elif row['Classification_target'] == 'BM':
#         df.loc[index, 'Classification_target'] = 5
#     elif row['Classification_target'] == 'BR':
#         df.loc[index, 'Classification_target'] = 6
#     elif row['Classification_target'] == 'NA':
#         df.loc[index, 'Classification_target'] = 7
#     elif row['Classification_target'] == 'Ace':
#         df.loc[index, 'Classification_target'] = 8
#     elif row['Classification_target'] == 'Dump':
#         df.loc[index, 'Classification_target'] = 9
#     elif row['Classification_target'] == 'Overpass':
#         df.loc[index, 'Classification_target'] = 10

# df.to_csv('numerical_df.csv', index=False)
# print(df['OPP_BR_Position'])
# df.head()
# print(df['Classification_target'])

This code will turn all the NaN values to -1, and then go through and bin each column so that I can run the CategoricalNB on it. Note that features with fewer than 5 unique values are dropped so that all the values can be binned. The data is then split into x and y DataFrames, with y being the target column.

In [70]:
df.fillna(-1, inplace=True)
for col in df.columns:
  if df[col].nunique() < 5:
    df.drop(col, axis=1, inplace=True)
def should_bin(column):
    return not all(value == 0 for value in column) and len(column.unique()) >= 5
df_binned = df.apply(lambda x: pd.qcut(x, q=[0, 1/5, 2/5, 3/5, 4/5, 1], labels=False, duplicates='drop') if should_bin(x) else x)
x = df_binned.drop('Classification_target', axis=1)
y = df_binned['Classification_target']

Here the model is ran 5 times and all training and test accuracies are reported, as well as the average for each.

In [75]:
train_accuracies = []
test_accuracies = []
for _ in range(5):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, stratify=y)
  model = CategoricalNB()
  model.fit(x_train, y_train)
  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)
  train_acc = accuracy_score(y_train, y_train_pred)
  test_acc = accuracy_score(y_test, y_test_pred)
  train_accuracies.append(train_acc)
  test_accuracies.append(test_acc)

train_avg = np.mean(train_accuracies)
test_avg = np.mean(test_accuracies)
train_accuracies.append(train_avg)
test_accuracies.append(test_avg)
results = pd.DataFrame({"Training Accuracy": train_accuracies, "Test Accuracy":test_accuracies})
results = results.rename({len(results)-1: 'Average'})
print(results)

         Training Accuracy  Test Accuracy
0                 0.648649       0.410714
1                 0.590090       0.500000
2                 0.608108       0.535714
3                 0.617117       0.428571
4                 0.572072       0.500000
Average           0.607207       0.475000
